In [1]:
!pip install jsonpath_ng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00


In [2]:
import os
import json
import numpy as np 
import pandas as pd 
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from jsonpath_ng import parse
from enum import Enum
%matplotlib inline 

# Source to Target Mapping - OOP Approach for Single Table

### Data

In [3]:

import json
with open("Glassdoor Gender.json", "r") as file:
    data = json.load(file)
    for item in data:
        item["id"] = str(item["id"])
data[:10]

[{'salary': 42363, 'id': '6789'},
 {'salary': 90208, 'id': '6790'},
 {'salary': 70890, 'id': '6791'},
 {'salary': 67585, 'id': '6792'},
 {'salary': 112976, 'id': '6793'},
 {'salary': 106524, 'id': '6794'},
 {'salary': 62759, 'id': '6795'},
 {'salary': 84007, 'id': '6796'},
 {'salary': 86220, 'id': '6797'},
 {'salary': 95584, 'id': '6798'}]

###  Abstract Base Calss (ABC) for mutual methods

In [4]:
class Interface(ABC):

    @abstractmethod
    def get_data_by_field(self, field_name):
        """Fetch the data by given feild name """

    @abstractmethod
    def get_data_by_id(self, id):
        """Fetch the data by given ID  """

    @abstractmethod
    def get(self):
        """Fetch all data """

### Transform Operations
inherithed from Enum - class that automatic enumrate the variables

In [6]:
class TransformMask(Enum):
    # add here any masks you want
    CLEAN_STRING = ".strip().lower()" 
    CAPITAL_LETTER = ".strip().lower().title()"
    CLEAN_NUM = ""
  

### Database Class - Define Common Properties for Source, Target, Mapping

In [7]:
class Database:
    def __init__(self):
        self.db = {
            "source": [],
            "destination": [],
            "transform": [],
            "mapping": []
        }
        self.add_source(1, "id", "str")
        self.add_destination(1, "id", "str")
        self.add_transform(1, "CLEAN_STRING")
        self.add_mapping(1, 1, 1, 1)
        self.add_source(2, "salary", "int")
        self.add_destination(2, "salary", "float")
        self.add_transform(2, "CLEAN_NUM")
        self.add_mapping(2, 2, 2, 2)

    
    def add_source(self, id, field, typ):
            self.db["source"].append({
                "id": id,
                "source_field_name": field,
                "source_field_mapping": field,
                "source_field_type": typ,
                "is_required": True,
            })
    
    def add_destination(self, id, field, typ):
        self.db["destination"].append({
            "id": id,
            "destination_field_name": field,
            "destination_field_mapping": field,
            "destination_field_type": typ,
            "default_value": "n/a", 
        })

    def add_transform(self, id, mask):
        self.db["transform"].append({
            "id": id,
            "transform_mask": mask,
        })

    def add_mapping(self, id, source, dest, transform):
      self.db["mapping"].append({
          "id": id,
          "mapping_source": source,
          "mapping_destination": dest,
          "mapping_transform": transform,
        })


    @property
    def get_data_source_target_mapping(self):
      return self.db


### Source class

Inherited from Interface for the common methods and from Database for common variables

In [8]:
class Source(Interface, Database):
    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("source")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id") == self.id:
                return x
        return None

### Target class

Inherited from Interface for the common methods and from Database for common variables

In [9]:
class Target(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("destination")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Transform Class

Inherited from Interface for the common methods and from Database for common variables

In [10]:
class Transform(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    # should be implemented - inherited from Interface
    def get_data_by_field(self, field_name):
        data = self.get
        for item in data:
            for key, value in item.items():
                if key == field_name:
                    return item
        return None

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("transform", [])

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

### Mapping class

Inherited from Interface for the common methods and from Database for common variables

In [11]:
class Mappings(Interface, Database):

    def __init__(self):
        Database.__init__(self)

    @property
    def get(self):
        return self.get_data_source_target_mapping.get("mapping")

    def get_data_by_id(self, id):
        self.id = id
        data = self.get
        for x in data:
            if x.get("id").__str__() == self.id.__str__():
                return x
        return None

    def get_data_by_field(self, field_name):
        return None

### Format Class - JSON

Search the source data value inside a JSON file 

In [12]:
class JsonQuery:
    def __init__(self, json_path, json_data):
        self.json_path = json_path
        self.json_data = json_data

    def get(self):
        jsonpath_expression = parse(self.json_path)
        match = jsonpath_expression.find(self.json_data)
        source_data_value = match[0].value
        return source_data_value

### Combine it All - STTM

In [13]:
class STTM:
    def __init__(self, input_json):
        self.input_json = input_json
        self.mapping_instance = Mappings()
        self.source_instance = Source()
        self.destination_instance = Target()
        self.transform_instance = Transform()
        self.look_up_mask = {i.name: i.value for i in TransformMask}
        self.json_data_transformed = {}

    def _get_mapping_data(self):
        return self.mapping_instance.get

    def _get_mapping_source_data(self):
        return self.source_instance.get

    def get_transformed_data(self):

        for mappings in self._get_mapping_data():

            """fetch the source mapping """
            mapping_source_id = mappings.get("mapping_source")
            mapping_destination_id = mappings.get("mapping_destination")
            mapping_transform_id = mappings.get("mapping_transform")

            mapping_source_data = self.source_instance.get_data_by_id(id=mapping_source_id)
            transform_data = self.transform_instance.get_data_by_id(id=mapping_transform_id)

            """Fetch Source  field Name"""
            source_field_name = mapping_source_data.get("source_field_name")

            """if field given is not present incoming json """
            if source_field_name not in self.input_json.keys():
                if mapping_source_data.get("is_required"):
                    raise Exception(
                        "Alert ! Field {} is not present in JSON please FIX mappings ".format(source_field_name))
                else:
                    pass

            else:
                source_data_value = JsonQuery(
                    json_path=mapping_source_data.get("source_field_mapping"),
                    json_data=self.input_json
                ).get()

                """check the data type for source if matches with what we have """
                if mapping_source_data.get("source_field_type") != type(source_data_value).__name__:
                    if source_data_value is not None:
                        _message = (
                            "Alert ! Source Field :{} Datatype has changed from {} to {} ".format(source_field_name,
                                                                                                  mapping_source_data.get(
                                                                                                      "source_field_type"),
                                                                                                  type(
                                                                                                      source_data_value).__name__))
                        print(_message)
                        raise Exception(_message)

                """Query and fetch the Destination | target """
                destination_mappings_json_object = self.destination_instance.get_data_by_id(
                    id=mappings.get("mapping_destination"))

                destination_field_name = destination_mappings_json_object.get("destination_field_name")
                destination_field_type = destination_mappings_json_object.get("destination_field_type")

                dtypes = [str, float, list, int, set, dict]

                for dtype in dtypes:

                    """Datatype Conversion """
                    if destination_field_type == str(dtype.__name__):

                        """is source is none insert default value"""
                        if source_data_value is None:
                            self.json_data_transformed[destination_field_name] = dtype.__call__(
                                destination_mappings_json_object.get("default_value")
                            )

                        else:
                            """check if you have items to transform"""
                            if transform_data is not None:
                                """ check for invalid mask name """
                                if transform_data.get("transform_mask") not in list(self.look_up_mask.keys()):
                                    raise Exception(
                                        f"Specified Transform {transform_data.get('transform_mask')} is not available please select from following Options :{list(self.look_up_mask.keys())}")
                                else:
                                    mask_apply = self.look_up_mask.get(transform_data.get("transform_mask"))
                                    converted_dtype = dtype.__call__(source_data_value)
                                    mask = f'converted_dtype{mask_apply}'
                                    curated_value = eval(mask)
                                    self.json_data_transformed[destination_field_name] = curated_value

                            else:
                                self.json_data_transformed[destination_field_name] = dtype.__call__(source_data_value)

        return self.json_data_transformed

In [14]:
transformed_data = []
for item in data:
    helper = STTM(input_json=item)
    response = helper.get_transformed_data()
    transformed_data.append(response)
    print(response)

{'id': '6789', 'salary': 42363.0}
{'id': '6790', 'salary': 90208.0}
{'id': '6791', 'salary': 70890.0}
{'id': '6792', 'salary': 67585.0}
{'id': '6793', 'salary': 112976.0}
{'id': '6794', 'salary': 106524.0}
{'id': '6795', 'salary': 62759.0}
{'id': '6796', 'salary': 84007.0}
{'id': '6797', 'salary': 86220.0}
{'id': '6798', 'salary': 95584.0}
{'id': '6799', 'salary': 73357.0}
{'id': '6800', 'salary': 88422.0}
{'id': '6801', 'salary': 99545.0}
{'id': '6802', 'salary': 92067.0}
{'id': '6803', 'salary': 72038.0}
{'id': '6804', 'salary': 71182.0}
{'id': '6805', 'salary': 112392.0}
{'id': '6806', 'salary': 80306.0}
{'id': '6807', 'salary': 74523.0}
{'id': '6808', 'salary': 113252.0}
{'id': '6809', 'salary': 84638.0}
{'id': '6810', 'salary': 70643.0}
{'id': '6811', 'salary': 74167.0}
{'id': '6812', 'salary': 102120.0}
{'id': '6813', 'salary': 97376.0}
{'id': '6814', 'salary': 89415.0}
{'id': '6815', 'salary': 88037.0}
{'id': '6816', 'salary': 71105.0}
{'id': '6817', 'salary': 62692.0}
{'id': '6

In [15]:
pd.DataFrame(transformed_data)

,id,salary
0,6789,42363.0
1,6790,90208.0
2,6791,70890.0
3,6792,67585.0
4,6793,112976.0
...,...,...
995,7784,102593.0
996,7785,108977.0
997,7786,92347.0
998,7787,97376.0
